In [ ]:
def ForwardPass(self, sample, slide):
    sample_time = sample[slide]
    layer = 0 
    weightsum = 0.0
    StateWeightSum = 0.0
    forwardout = 0.0 
    
    for row in range(0, self.Top[0]):
        self.InlayerOutL0[slide+1][row] = sample_time[row]

    for y in range(0, self.Top[1]):
        for x in range(0, self.Top[0]):
            weightsum += self.InlayerOutL0[slide+1][x] * self.W1[x,y]

        for x in range(0, self.Top[1]):
            StateWeightSum += self.OutputSlideL1[slide][x] * self.StateW[x,y]
            
        forwardout = (weightsum + StateWeightSum) - self.B1[y]

        self.OutputSlideL1[slide+1][y] = self.sigmoid(forwardout)
        
        weightsum = 0
        StateWeightSum = 0

    layer = 1 #   hidden layer to output
    weightsum = 0.0
    #print(self.out, end=' ')
    
    for y in range(0, self.Top[layer+1]):
        for x in range(0, self.Top[layer]):
            weightsum += self.OutputSlideL1[slide+1][x] * self.W2[x,y]
            forwardout = (weightsum - self.B2[y])
            
        self.OutputSlideL2[slide+1][y] = self.sigmoid(forwardout) 
        
        weightsum = 0.0
        StateWeightSum = 0.0


def BPTT(self):  
    for i,sample in enumerate(self.Train_x):
        self.StateOut = np.ones(self.Top[1])
        self.ErL1 = np.zeros((len(sample)+1, self.Top[1]))   # need to modify for multiple layers
        self.OutputSlideL1 = np.zeros((len(sample)+1, self.Top[1]))
        
        for x in range(0,self.Top[1]):
            self.OutputSlideL1[0][x] = self.StateOut[x]
            
        self.InlayerOutL0 = np.zeros((len(sample)+1, self.Top[0]))
        self.OutputSlideL2 = np.zeros((len(sample)+1, self.Top[2]))
        
        for slide in range(0, len(sample)):
            self.ForwardPass(sample, slide)
            
        for slide in range(len(sample),0,-1):
            self.BackwardPass(sample, self.learn_rate, self.Train_y[i-1], slide)

In [1]:
# one to many RNN model

import numpy as np

np.random.seed(0)

class RecurrentNetwork(object):
    """When we say W_hh, it means a weight matrix that accepts a hidden state and produce a new hidden state.
    Similarly, W_xh represents a weight matrix that accepts an input vector and produce a new hidden state. This
    notation can get messy as we get more variables later on with LSTM and I simplify the notation a little bit in
    LSTM notes.
    """
    def __init__(self):
        self.hidden_state = np.zeros((3, 3))
        self.W_hh = np.random.randn(3, 3)
        self.W_xh = np.random.randn(3, 3)
        self.W_hy = np.random.randn(3, 3)
        self.Bh = np.random.randn(3,)
        self.By = np.random.rand(3,)

    def forward_prop(self, x):
        # The order of which you do dot product is entirely up to you.
        # The gradient updates will take care itself as long as the matrix dimension matches up.
        self.hidden_state = np.tanh(np.dot(self.hidden_state, self.W_hh) + np.dot(x, self.W_xh) + self.Bh)
        
        return self.W_hy.dot(self.hidden_state) + self.By


input_vector = np.ones((3, 3))
print(input_vector, ' input vec')
silly_network = RecurrentNetwork()

# Notice that same input, but leads to different ouptut at every single time step.
print(silly_network.forward_prop(input_vector))
print(silly_network.forward_prop(input_vector))
print(silly_network.forward_prop(input_vector))

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]  input vec
[[-1.73665315 -2.40366542 -2.72344361]
 [ 1.61591482  1.45557046  1.13262256]
 [ 1.68977504  1.54059305  1.21757531]]
[[-2.15023381 -2.41205828 -2.71701457]
 [ 1.71962883  1.45767515  1.13101034]
 [ 1.80488553  1.542929    1.21578594]]
[[-2.15024751 -2.41207375 -2.720968  ]
 [ 1.71963227  1.45767903  1.13200175]
 [ 1.80488935  1.54293331  1.21688628]]
